In [16]:
"""
Initial setup:
1) create virtualenv project using pycharm

2) install the following libraries to virtualenv:
pip install numpy
pip install pandas
pip install requests
pip install beautifulsoup4
pip install lxml
"""

'\nInitial setup:\n1) create virtualenv project using pycharm\n\n2) install the following libraries to virtualenv:\npip install numpy\npip install pandas\npip install requests\npip install beautifulsoup4\npip install lxml\n'

In [9]:
import requests  # library to send requests to web site(krisha.kz)
from bs4 import BeautifulSoup as bs  # library to copy all html-code
import time
import math
import re
import json
from lxml import html
from lxml import etree
import regex
import pandas as pd

In [11]:
headers = {
    'accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

session = requests.Session()

def get_all_urls(catalog_page_url):
    result = [catalog_page_url]
    url = catalog_page_url
    all_data = get_page(url)
    pages = math.ceil(all_data["total"] / all_data["limit"])
    for i in range(2, pages):
        url = catalog_page_url + '?page={i}'.format(i=i)
        if url not in result:
            result.append(url)
    return result


def get_page(url):
    request = session.get(url, headers=headers)
    tree = html.fromstring(request.content, parser=etree.HTMLParser(encoding="utf-8"))
    script = tree.xpath('//script[contains(., "BACKEND.components.catalogGrid")]/text()')[0]
    data = regex.compile(r"\{(?:[^{}]|(?R))*\}").findall(script)[0]
    all_data = json.loads(data)
    return all_data


def parse_kaspi_pages(urls):
    all_cards = []
    c = 1
    for url in urls:
        time.sleep(0.7)
        print("parsing: " + url)
        all_data = get_page(url)
        cards = all_data["cards"]
        for card in cards:
            card.pop("monthlyInstallment", None)
            card.pop("previewImages", None)
            card.pop("category", None)
            card.pop("unit", None)
            card["price"] = card["unitPrice"]
            card.pop("unitPrice")
            title_parts = card["title"].split(" ")
            card["color"] = title_parts[-1]
            card['full_name'] = ' '.join(title_parts[:-1])

        all_cards += cards
        c += 1
        if c > 1000: # abort if there is too many pages
            break
    return all_cards

base_url = 'https://kaspi.kz/shop/c/notebooks/'
items = parse_kaspi_pages(urls = get_all_urls(base_url))
session.close()
df = pd.json_normalize(items)
df.to_csv('kaspi_more_data.csv', encoding='utf-8', index=False)

parsing: https://kaspi.kz/shop/c/notebooks/
parsing: https://kaspi.kz/shop/c/notebooks/?page=2
parsing: https://kaspi.kz/shop/c/notebooks/?page=3
parsing: https://kaspi.kz/shop/c/notebooks/?page=4
parsing: https://kaspi.kz/shop/c/notebooks/?page=5
parsing: https://kaspi.kz/shop/c/notebooks/?page=6
parsing: https://kaspi.kz/shop/c/notebooks/?page=7
parsing: https://kaspi.kz/shop/c/notebooks/?page=8
parsing: https://kaspi.kz/shop/c/notebooks/?page=9
parsing: https://kaspi.kz/shop/c/notebooks/?page=10
parsing: https://kaspi.kz/shop/c/notebooks/?page=11
parsing: https://kaspi.kz/shop/c/notebooks/?page=12
parsing: https://kaspi.kz/shop/c/notebooks/?page=13
parsing: https://kaspi.kz/shop/c/notebooks/?page=14
parsing: https://kaspi.kz/shop/c/notebooks/?page=15
parsing: https://kaspi.kz/shop/c/notebooks/?page=16
parsing: https://kaspi.kz/shop/c/notebooks/?page=17
parsing: https://kaspi.kz/shop/c/notebooks/?page=18
parsing: https://kaspi.kz/shop/c/notebooks/?page=19
parsing: https://kaspi.kz/sh